In [1]:
import re
from pathlib import Path
from os import system, name

In [2]:
def clear():
    if name == 'nt':
        system('cls')
    else:
        system('clear')

In [3]:
def tr3(string, lista): # conversão e filtragem
    stringnova=""
    tam=len(string)
    x=0
    while x<tam:
        if(x<tam-1 and str(string[x]) + str(string[x+1]) in lista):
            stringnova+= str(" ")+str(string[x])+str(string[x+1])+str(" ")
            x+=1
        elif(str(string[x]) in lista):
            stringnova+=" "+str(string[x])+" "
        else:
            stringnova+=str(string[x])
        x+=1
    return list(filter(lambda x:x!="",stringnova.split(" ")))

In [4]:
class Grammar:
    def __init__(self, rules):
        rules = tuple(rules)
        self.rules = tuple(self._parse(rule) for rule in rules)

    def _parse(self, rule):
        return tuple(rule.replace(' ', '').split('->'))
        
    @staticmethod 
    def is_nonterminal(symbol):
        return symbol.isalpha() and symbol.isupper()

    @property
    def nonterminals(self):
        return set([nt for nt, _ in self.rules])
        
    @property
    def terminals(self):       
        a = [
            expression
            for _, expression in self.rules
            if not self.is_nonterminal(expression)
        ]
        
        nt = sorted([nt for nt, _ in self.rules], key=len, reverse=True)
        a = [i for j in [tr3(i, nt) for i in a] for i in j]

        for i in nt: 
            for j in range(0, len(a)):
                if re.search(i, a[j]):
                    a[j] = a[j].replace(i, '')
                a[j] = " ".join(a[j].split())
        
        tmp = []  
        for i in a:
            if i != '' and i not in tmp:
                tmp.append(i)
        a = tmp

        tmp = []
        for i in a:
            if len(i) > 1 and not i.isalpha():
                tmp2 = []
                tmp2.append(list(i))
                tmp2 = [i for j in tmp2 for i in j]
                for j in tmp2:
                    tmp.append(j)
            else:
                tmp.append(i)
        a = tmp
        tmp = [] 
        # a = [i for j in [list(i) for i in a] for i in j] # Lugar Onde Comentar ou Descomentar De Acordo com a gramatica
        return set(a) 

In [5]:
def transform2(string,lista):
    listanova=[]
    texto=""
    tam=len(string)
    x = 0
    while x < tam:
        texto +=string[x]
        if(x < tam-1 and texto+string[x+1] in lista):
            listanova.append(texto+string[x+1])
            texto=""
            x +=1
        elif texto in lista: 
            listanova.append(texto)
            texto=""
        x+=1
    return listanova

In [6]:
def union(first, begins):
    n = len(first)
    first |= begins
    return len(first) != n

In [7]:
def FirstAndFollow(terminais, nao_terminais, regras):
    # lista dos terminais e não terminais
    lista = sorted((list(terminais)+list(nao_terminais)),
                   key=len, reverse=True)

    first = {i: set() for i in nao_terminais} # Conjunto de inicio do First
    first.update((i, {i}) for i in terminais)
    follow = {i: set() for i in nao_terminais} # Conjunto de inicio do Follow


    # regras gramaticais separados por tuplas (Não terminais, expressões)
    rule = tuple([(i, transform2(j, lista)) for i, j in regras])

    if not ('^' in rule[0][0]):
        rule = [list(i) for i in rule]
        valor = [rule[0][0], '$']
        rule.insert(0, ['^', valor])
        lista = sorted((list(lista) + list('$') + list('^')),
                       key=len, reverse=True)
        terminais |= {'$'}
        nao_terminais |= {'^'}
        rule = tuple(tuple(i) for i in rule)
        first = {i: set() for i in nao_terminais}
        first.update((i, {i}) for i in terminais)
        follow = {i: set() for i in nao_terminais}

    #Determina o simbolo do Epsilon
    epsilon = {'ε'}

    # Algoritmo Não Recursivo
    # O loop de parada 
    # Quando acaba de processar todas as expressões 
    # e ponto de retorno da função
    # quando o updated for True

    while True:
        updated = False
        for nt, expression in rule: #Processa expressão por expressão por vez
            # Logica que Calcula o First da Expressão
            # Identifica em quais expressões está o epsilon
            # e armazena o simbolo não terminal correspondente
            # a expressão no conjunto Epsilon
            for symbol in expression:
                updated |= union(first[nt], first[symbol])
                if symbol not in epsilon:
                    break
                else:
                    updated |= union(epsilon, {nt})

            #Logica que Calcula o Follow da Expressão
            aux = follow[nt]
            for symbol in reversed(expression):
                if symbol in follow:
                    updated |= union(follow[symbol], aux)
                if symbol in epsilon:
                    # União do conjunto first[symbol] com o 
                    # conjunto auxiliar anterior 
                    # formando um novo auxiliar
                    aux = aux.union(first[symbol])
                else:
                    aux = first[symbol]

        if not updated:
            for chave, valor in follow.items():
                if 'ε' in follow[chave]:
                    follow[chave] = follow[chave] - {'ε'}
            cond1 = False
            cond2 = False
            for i in epsilon:
                if '^' in i:
                    cond1 = True
                if 'ε' in i:
                    cond2 = True

            for i in nao_terminais:
                if '^' in i:
                    first.pop(i)
                    follow.pop(i)
                if cond1 and '^' in i:
                    epsilon.remove(i)

            for i in terminais:
                # first.pop(i)
                first['$'] = '$'
                if cond2 and 'ε' in i:
                    epsilon.remove(i)
            for i in {'$'}:
                first.pop(i)

            return first, follow, epsilon

In [8]:
def tratarArq(lista): 
    lista = " ".join(lista.split())
    lista = re.split(r' ', lista)
    return lista

In [9]:
def lerArquivo(dir):
    arquivo = []
    b = []
    inicio = ""
    fim = ""
    
    with open(dir, "r") as gramatica:
        for line in gramatica:
            arquivo.append(line.replace('Îµ', 'ε').strip().split('\n'))
    arquivo = [i for j in arquivo for i in j]

    for i in range(0, len(arquivo)):
        temp = []
        temp = tratarArq(arquivo[i])
        arquivo[i] = temp
             
    for i in range(0, len(arquivo)):
        tam = len(arquivo[i])
        for j in range(0, tam):    
            if "|" in arquivo[i][j]:
                inicio = arquivo[i][0]+" "+arquivo[i][1]
                fim = arquivo[i][-1]
                suma = inicio+" "+fim
                arquivo.append(tratarArq(suma))
    
    for i in arquivo:
        b.append(" ".join(i[:3]))
    return b

In [10]:
def gravarArquivo(first, follow, epsilon):
    outpath = 'output'
    Path(outpath).mkdir(exist_ok=True)
    with open(outpath+'/First.txt', 'w+', encoding="utf-8") as file:
        a = "Epsilon na Gramatica = {}".format(epsilon)
        file.write(a + '\n')
        for i in first:
            a = "Primeiro({}) = {}".format(i, first[i])
            file.write(a + '\n')
    print("\n")
    with open(outpath+'/Follow.txt', 'w+', encoding="utf-8") as file:
        a = ''
        for i in follow:
            a = "Follow({}) = {}".format(i, follow[i])
            file.write(a + '\n')

In [11]:
def printArquivo(dir):
    with open(dir, "r", encoding="utf-8") as file:
        for line in file:
            print(line)

In [12]:
def limpar_terminais_first(terminais, first):
    for i in terminais - {'$'}:
      first.pop(i)

In [13]:
def visualizar_saida():
    print('-----------------------------------------------------------------')
    X = True
    while X:
        op = input('1 - Visualizar First\n2 - Visualizar Follow\n3 - Visualizar First e Follow\n\nDigite a Opção: ')
        print('\n---------------------------------------------------------------------\n')
        if op == '1':
            printArquivo('output/First.txt')
            X = False
        if op == '2':
            printArquivo('output/Follow.txt')
            X = False
        if op == '3':
            printArquivo('output/First.txt')
            print('-----------------------------------------------------------------\n')
            printArquivo('output/Follow.txt')
            X = False

In [14]:
def visualizar_gr(lista):
    for i in lista:
        print(i)

In [15]:
def comentarios_sobre_gramaticas(escolha):
    if escolha == 1:
        print("Sobre a Gramatica a.txt, é necessário (((comentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 2:
        print("Sobre a Gramatica b.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 3:
        print("Sobre a Gramatica c.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 4:
        print("Sobre a Gramatica d.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 5:
        print("Sobre a Gramatica e.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 6:
        print("Sobre a Gramatica f.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")
    if escolha == 7:
        print("Sobre a Gramatica g.txt, é necessário (((descomentar))) \na penuntima linha da Função 'terminals' \nLocalizada em grammar.py Dentro da Classe Grammar \npara um bom funcionamento do código")


In [19]:
def main():
    dir = "gramaticas"

    gramatica = [dir+'/a.txt(principal)', dir+'/b.txt', dir+'/c.txt', dir+'/d.txt', dir+'/e.txt', dir+'/f.txt', dir+'/g.txt']
    
    clear()
    # print("Selecione Entre 1 e 9:\n")
    # print("1 - a.txt\n2 - b.txt\n3 - c.txt\n4 - d.txt\n5 - e.txt\n6 - f.txt\n7 - g.txt\n8 - h.txt\n9 - i.txt\n\n")
    escolha = int(input('Selecione Entre 1 e 9:\n1 - a.txt\n2 - b.txt\n3 - c.txt\n4 - d.txt\n5 - e.txt\n6 - f.txt\n7 - g.txt\n8 - h.txt\n9 - i.txt\n\nEscolha uma Gramatica: '))

    if not (escolha in [i for i in range(1, 10)]):
        print("Escolha Invalida")
        exit()
    
    clear()

    comentarios_sobre_gramaticas(escolha)
    confirmar = input("Digite OK para Continuar ou qualquer coisa para finalizar: ")
    if confirmar != 'OK':
        clear()
        exit()
    elif confirmar == 'OK':
        pass
    
    clear()

    gr = lerArquivo(gramatica[escolha-1])

    visualizar_gr(gr)

    gramatica = Grammar(gr)

    terminais, nao_terminais, regras = gramatica.terminals, gramatica.nonterminals, gramatica.rules

    first, follow, epsilon = FirstAndFollow(terminais, nao_terminais, regras)

    limpar_terminais_first(terminais, first)

    gravarArquivo(first, follow, epsilon)

    visualizar_saida()

In [20]:
main()

Sobre a Gramatica a.txt, é necessário (((comentar))) 
a penuntima linha da Função 'terminals' 
Localizada em grammar.py Dentro da Classe Grammar 
para um bom funcionamento do código
^ -> E$
E -> TE'
E' -> +TE'
T -> FT'
T' -> *FT'
F -> (E)
E' -> ε
T' -> ε
F -> id


-----------------------------------------------------------------

---------------------------------------------------------------------

Epsilon na Gramatica = {"T'", "E'"}

Primeiro(T') = {'ε', '*'}

Primeiro(E') = {'ε', '+'}

Primeiro(E) = {'id', '('}

Primeiro(F) = {'id', '('}

Primeiro(T) = {'id', '('}

-----------------------------------------------------------------

Follow(T') = {')', '$', '+'}

Follow(E') = {')', '$'}

Follow(E) = {')', '$'}

Follow(F) = {')', '$', '*', '+'}

Follow(T) = {')', '$', '+'}



In [18]:
# Anotações
# Logica 
# (('E', "TE'"), ("E'", "+TE'"), ('T', "FT'"), ("T'", "*FT'"), ('F', '(E)'), ("E'", 'ε'), ("T'", 'ε'), ('F', 'id'))

# [['E', "TE'"], ["E'", "+TE'"], ['T', "FT'"], ["T'", "*FT'"], ['F', '(E)'], ["E'", 'ε'], ["T'", 'ε'], ['F', 'id']]

# [['E', ["T","E'"]], ["E'", ["+","T","E'"]], ['T', ["F","T'"]], ["T'", ["*","F","T'"]], ['F', ["(","E",")"]], ["E'", ['ε']], ["T'", ['ε']], ['F', ['id']]]

# (('E', ["T","E'"]), ("E'", ["+","T","E'"]), ('T', ["F","T'"]), ("T'", ["*","F","T'"]), ('F', ["(","E",")"]), ("E'", ['ε']), ("T'", ['ε']), ('F', ['id']) )